In [1]:
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
list_of_gpus = tf.config.list_physical_devices('GPU')
if len(list_of_gpus) > 0:
    print(f"GPUs detected! {len(list_of_gpus)} GPUs being used")
else:
    print("no GPUs detected =(")

GPUs detected! 1 GPUs being used


2022-09-22 15:52:54.929720: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 15:52:54.974819: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 15:52:54.975167: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
wandb.init(project="wandb-notebook")

wandb: Currently logged in as: runai-jcosme. Use `wandb login --relogin` to force relogin


In [4]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 15,
  "batch_size": 128
}

In [5]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

2022-09-22 15:53:05.453219: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-22 15:53:05.453946: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 15:53:05.454302: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 15:53:05.454633: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

In [7]:
opt = keras.optimizers.Adam(
    learning_rate=wandb.config['learning_rate']
)
model.compile(
    loss="categorical_crossentropy", 
    optimizer=opt, 
    metrics=["accuracy"]
)



In [8]:
model.fit(
    x_train,
    y_train, 
    validation_split=0.1, 
    batch_size=wandb.config['batch_size'],
    epochs=wandb.config['epochs'],
    callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/15


2022-09-22 15:53:12.508056: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 15:53:12.508338: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-09-22 15:53:12.508480: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-09-22 15:53:12.509071: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 15:53:12.509367: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 15:53:12.509600: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] su

422/422 [==============================] - ETA: 0s - loss: 0.3653 - accuracy: 0.8893

2022-09-22 15:53:17.712452: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


...

422/422 [==============================] - 2s 5ms/step - loss: 0.0417 - accuracy: 0.9871 - val_loss: 0.0313 - val_accuracy: 0.9918
Epoch 11/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0396 - accuracy: 0.9876 - val_loss: 0.0366 - val_accuracy: 0.9902
Epoch 12/15
419/422 [============================>.] - ETA: 0s - loss: 0.0383 - accuracy: 0.9880INFO:tensorflow:Assets written to: /home/jovyan/work/projects/wandb_demo/wandb/run-20220922_155257-3owblua5/files/model-best/assets


wandb: Adding directory to artifact (/home/jovyan/work/projects/wandb_demo/wandb/run-20220922_155257-3owblua5/files/model-best)... Done. 0.0s


422/422 [==============================] - 2s 6ms/step - loss: 0.0382 - accuracy: 0.9880 - val_loss: 0.0299 - val_accuracy: 0.9925
Epoch 13/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0362 - accuracy: 0.9882 - val_loss: 0.0301 - val_accuracy: 0.9913
Epoch 14/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0332 - accuracy: 0.9892 - val_loss: 0.0326 - val_accuracy: 0.9918
Epoch 15/15
420/422 [============================>.] - ETA: 0s - loss: 0.0332 - accuracy: 0.9889INFO:tensorflow:Assets written to: /home/jovyan/work/projects/wandb_demo/wandb/run-20220922_155257-3owblua5/files/model-best/assets


wandb: Adding directory to artifact (/home/jovyan/work/projects/wandb_demo/wandb/run-20220922_155257-3owblua5/files/model-best)... Done. 0.0s


422/422 [==============================] - 2s 5ms/step - loss: 0.0332 - accuracy: 0.9889 - val_loss: 0.0279 - val_accuracy: 0.9922
